# TODO

- Check if PS are weighted by fractionality
- Add type 3 and 4 dynamic features
- Add the missing six features in type 7 dynamic features

# Checklist

- [ ] Set number of threads to one
- [ ] Turn-off primal heuristics
- [ ] Set optimal objective value as the primal bound
- [ ] Disable cuts after root node
- [ ] Limit dual simplex LP iterations to 50 for calculating SB score

In [1]:
from math import floor
from operator import itemgetter

import cplex as CPX
import cplex.callbacks as CPX_CB
import numpy as np
from scipy.sparse import csr_matrix

In [2]:
# Number of nodes to run strong branching
THETA = 500

# Maximum number of candidates to calculate strong-branching scores
K = 10

# Threshold
EPSILON = 1e-6

In [3]:
MINIMIZE = 1

OPTIMAL = 1
INFEASIBILITY = 1e-6
UPPER_BOUND = 'U'
LOWER_BOUND = 'L'

BS_DEFAULT = 1
BS_SB = 2
BS_PC = 3
BS_SB_PC = 4
BS_SB_ML_SVMRank = 5
BS_SB_ML_LR = 6
BS_SB_ML_NN = 7
BS_SB_ML_GNN = 8

In [4]:
def set_params(c, primal_bound, test=False):
    # Single threaded
    c.parameters.threads.set(1)

    # Disable primal heuristics
    c.parameters.mip.strategy.heuristiceffort.set(0)    
        
    # Set the primal bound if provided    
    if primal_bound is not None:
        if c.objective.get_sense() == MINIMIZE:
            c.parameters.mip.tolerances.lowercutoff.set(primal_bound)
        else:
            c.parameters.mip.tolerances.uppercutoff.set(primal_bound)
            
    if not test:
        disable_output(c)

def disable_cuts(c):        
    c.parameters.mip.limits.eachcutlimit.set(0)
    c.parameters.mip.cuts.bqp.set(-1)
    c.parameters.mip.cuts.cliques.set(-1)
    c.parameters.mip.cuts.covers.set(-1)
    c.parameters.mip.cuts.disjunctive.set(-1)
    c.parameters.mip.cuts.flowcovers.set(-1)
    c.parameters.mip.cuts.pathcut.set(-1)
    c.parameters.mip.cuts.gomory.set(-1)
    c.parameters.mip.cuts.gubcovers.set(-1)
    c.parameters.mip.cuts.implied.set(-1)
    c.parameters.mip.cuts.localimplied.set(-1)
    c.parameters.mip.cuts.liftproj.set(-1)
    c.parameters.mip.cuts.mircut.set(-1)
    c.parameters.mip.cuts.mcfcut.set(-1)
    c.parameters.mip.cuts.rlt.set(-1)
    c.parameters.mip.cuts.zerohalfcut.set(-1)
    
def disable_output(c):        
    c.set_log_stream(None)
    c.set_error_stream(None)
    c.set_warning_stream(None)
    c.set_results_stream(None)
    
def solve_as_lp(c, max_iterations=None):
    disable_output(c)
    c.set_problem_type(c.problem_type.LP)
    # Set the maximum number of iterations for solving the LP
    if max_iterations is not None:
        c.parameters.simplex.limits.iterations = max_iterations
        
    c.solve()
    status = c.solution.get_status()
    objective = c.solution.get_objective_value() if status == OPTIMAL else INFEASIBILITY
    return status, objective
    
def apply_branch_history(c, branch_history):
    for b in branch_history:
        b_var = b[0]
        b_type = b[1]
        b_val = b[2]

        if b_type == LOWER_BOUND:
            c.variables.set_lower_bounds(b_var, b_val)
        elif b_type == UPPER_BOUND:
            c.variables.set_upper_bounds(b_var, b_val)

In [5]:
# https://github.com/raviagrwl420/variable-selection/blob/master/src/static_features.py

class StaticFeatures:
    def __init__(self, cplex_instance):
        self.num_vars = cplex_instance.variables.get_num()
        self.var_names = cplex_instance.variables.get_names() # Simple list of strings
        self.var_types = cplex_instance.variables.get_types() # Simple list of 'B', 'I' or 'C'
        self.columns = cplex_instance.variables.get_cols() # List of SparsePair objects - SparsePair(ind=[], val=[])
        self.num_rows = cplex_instance.linear_constraints.get_num()
        self.rows = cplex_instance.linear_constraints.get_rows() # List of SparsePair objects - SparsePair(ind=[], val=[])
        self.rhs = np.array(cplex_instance.linear_constraints.get_rhs()) # Simple list of values
        self.obj = np.array(cplex_instance.objective.get_linear())

        # Generate the row x column matrix
        self.matrix = csr_matrix((self.num_rows, self.num_vars))
        for i, row in enumerate(self.rows):
            data = np.array(row.val)
            row_indices = np.empty(len(row.ind))
            row_indices.fill(i)
            col_indices = np.array(row.ind)
            self.matrix += csr_matrix((data, (row_indices, col_indices)), shape=(self.num_rows, self.num_vars))

        ## Part 1: Objective function coefficients
        # 1. Raw obj function coefficients
        raw = self.obj.copy().reshape(-1, 1)

        # 2. Positive only obj function coefficients
        positive_only = self.obj.copy().reshape(-1, 1)
        positive_only[positive_only < 0] = 0

        # 3. Negative only obj function coefficients
        negative_only = self.obj.copy().reshape(-1, 1)
        negative_only[negative_only > 0] = 0

        # Add 1, 2, 3 to features
        self.features = np.c_[raw, positive_only, negative_only]

        ## Part 2: Num. Constraints
        # 4. Number of constraints that the variable participates in (with a non-zero coefficient)
        non_zeros = self.matrix != 0
        num_const_for_var = np.transpose(non_zeros.sum(0))

        # Add 4 to features
        self.features = np.c_[self.features, num_const_for_var]

        ## Part 3: Stats for constraint degrees
        num_var_for_const = non_zeros.sum(1)
        degree_matrix = non_zeros.multiply(csr_matrix(num_var_for_const)).todense()

        # 5. Mean of degrees
        mean_degrees = np.transpose(np.mean(degree_matrix, axis=0))

        # 6. Stdev of degrees
        std_degrees = np.transpose(np.std(degree_matrix, axis=0))

        # 7. Min of degrees
        min_degrees = np.transpose(np.min(degree_matrix, axis=0))

        # 8. Max of degrees
        max_degrees = np.transpose(np.max(degree_matrix, axis=0))

        # Add 5, 6, 7, 8 to features
        self.features = np.c_[self.features, mean_degrees, std_degrees, min_degrees, max_degrees]

        ## Part 4: Stats for constraint coefficients
        pos_coeffs = self.matrix > 0
        pos_matrix = self.matrix.todense()
        pos_matrix[pos_matrix < 0] = 0
        neg_coeffs = self.matrix < 0
        neg_matrix = self.matrix.todense()
        neg_matrix[neg_matrix > 0] = 0

        # 9, 10, 11, 12, 13
        count_pos_coeffs = np.transpose(pos_coeffs.sum(0))
        mean_pos_coeffs = np.transpose(np.mean(pos_matrix, axis=0))
        std_pos_coeffs = np.transpose(np.std(pos_matrix, axis=0))
        min_pos_coeffs = np.transpose(np.min(pos_matrix, axis=0))
        max_pos_coeffs = np.transpose(np.max(pos_matrix, axis=0))

        # Add 9, 10, 11, 12, 13 to features
        self.features = np.c_[self.features, count_pos_coeffs, mean_pos_coeffs, std_pos_coeffs, min_pos_coeffs, max_pos_coeffs]

        # 14, 15, 16, 17, 18
        count_neg_coeffs = np.transpose(neg_coeffs.sum(0))
        mean_neg_coeffs = np.transpose(np.mean(neg_matrix, axis=0))
        std_neg_coeffs = np.transpose(np.std(neg_matrix, axis=0))
        min_neg_coeffs = np.transpose(np.min(neg_matrix, axis=0))
        max_neg_coeffs = np.transpose(np.max(neg_matrix, axis=0))

        # Add 14, 15, 16, 17, 18 to features
        self.features = np.c_[self.features, count_neg_coeffs, mean_neg_coeffs, std_neg_coeffs, min_neg_coeffs, max_neg_coeffs]

        # Add additional data for dynamic features
        self.sum_coeffs = np.sum(self.matrix, axis=1)
        self.sum_pos_coeffs = np.sum(pos_matrix, axis=1)
        self.sum_neg_coeffs = np.sum(neg_matrix, axis=1)
        self.pos_coeff_matrix = csr_matrix(pos_matrix)
        self.neg_coeff_matrix = csr_matrix(neg_matrix)

In [6]:
# Reference: https://github.com/raviagrwl420/variable-selection/blob/master/src/dynamic_features.py

class DynamicFeatures:
    def __init__(self, branch_instance, static_features, candidates):
        # Part 1: Slack and ceil distances
        self.values = np.array(branch_instance.get_values()).reshape(-1, 1)
        self.values = self.values[candidates] # Filter by candidates
        
        # 1. Min of slack and ceil
        ceil = np.ceil(self.values)
        floor = np.floor(self.values)
        fractionality = np.minimum(self.values - floor, ceil - self.values)

        # 2. Distance from ceil
        dist_ceil = ceil - self.values

        # Add 1, 2 to features
        self.features = np.c_[fractionality, dist_ceil]

        # Part 2: Pseudocosts

        # 3. Upwards and downwards pseudocosts weighted by fractionality
        self.pseudocosts = np.array(branch_instance.get_pseudo_costs())
        self.pseudocosts = self.pseudocosts[candidates]
        up_down_pc = self.pseudocosts * fractionality
        
        # 4. Sum of pseudocosts weighted by fractionality
        sum_pc = np.sum(self.pseudocosts, axis=1).reshape(-1, 1) * fractionality

        # 5. Ratio of pseudocosts weighted by fractionality
        ratio_pc = (self.pseudocosts[:, 0] / self.pseudocosts[:, 1]).reshape(-1, 1) * fractionality
        ratio_pc[np.isnan(ratio_pc)] = 0
        ratio_pc[np.isinf(ratio_pc)] = 0

        # 6. Prod of pseudocosts weighted by fractionality
        prod_pc = np.prod(self.pseudocosts, axis=1).reshape(-1, 1) * fractionality

        # Add 3, 4, 5, 6 to features
        self.features = np.c_[self.features, up_down_pc, sum_pc, ratio_pc, prod_pc]
        
        # print('1, 2', self.features.shape)

        # Skipped Parts 3 and 4
        # Part 5: Min/max ratios of constraint coeffs to RHS
        rhs = static_features.rhs.reshape(-1, 1)
        pos_rhs = rhs[rhs > 0]
        neg_rhs = rhs[rhs < 0]

        mat = static_features.matrix.todense()
        candidate_matrix = static_features.matrix[:, candidates].todense()
        pos_ratio_matrix = np.divide(candidate_matrix[(rhs > 0).ravel(), :], pos_rhs.reshape(-1, 1))
        pos_ratio_matrix = pos_ratio_matrix if pos_ratio_matrix.size else np.zeros((1, candidate_matrix.shape[1]))
        neg_ratio_matrix = np.divide(candidate_matrix[(rhs < 0).ravel(), :], neg_rhs.reshape(-1, 1))
        neg_ratio_matrix = neg_ratio_matrix if neg_ratio_matrix.size else np.zeros((1, candidate_matrix.shape[1]))

        # 7. Min ratio for positive RHS
        min_ratio_pos = np.transpose(np.min(pos_ratio_matrix, axis=0))
        
        # 8. Max ratio for positive RHS
        max_ratio_pos = np.transpose(np.max(pos_ratio_matrix, axis=0))

        # 9. Min ratio for negative RHS
        min_ratio_neg = np.transpose(np.min(neg_ratio_matrix, axis=0))

        # 10. Max ratio for negative RHS
        max_ratio_neg = np.transpose(np.max(neg_ratio_matrix, axis=0))

        # Add 7, 8, 9, 10 to features
        self.features = np.c_[self.features, min_ratio_pos, max_ratio_pos, min_ratio_neg, max_ratio_neg]
        # print('1, 2, 5', self.features.shape)
        
        # Part 6: Min/max for one-to-all coefficient ratios
        pos_coeff_matrix = static_features.pos_coeff_matrix[:, candidates]
        neg_coeff_matrix = static_features.neg_coeff_matrix[:, candidates]

        sum_pos_coeffs = static_features.sum_pos_coeffs
        sum_neg_coeffs = static_features.sum_neg_coeffs

        pos_pos_ratio_matrix = pos_coeff_matrix.todense() / sum_pos_coeffs
        pos_pos_ratio_matrix[np.isnan(pos_pos_ratio_matrix)] = 0
        pos_pos_ratio_matrix[np.isinf(pos_pos_ratio_matrix)] = 0
        pos_neg_ratio_matrix = pos_coeff_matrix.todense() / sum_neg_coeffs
        pos_neg_ratio_matrix[np.isnan(pos_neg_ratio_matrix)] = 0
        pos_neg_ratio_matrix[np.isinf(pos_neg_ratio_matrix)] = 0
        neg_neg_ratio_matrix = neg_coeff_matrix.todense() / sum_neg_coeffs
        neg_neg_ratio_matrix[np.isnan(neg_neg_ratio_matrix)] = 0
        neg_neg_ratio_matrix[np.isinf(neg_neg_ratio_matrix)] = 0
        neg_pos_ratio_matrix = neg_coeff_matrix.todense() / sum_pos_coeffs
        neg_pos_ratio_matrix[np.isnan(neg_pos_ratio_matrix)] = 0
        neg_pos_ratio_matrix[np.isinf(neg_pos_ratio_matrix)] = 0        

        pos_pos_ratio_min = np.transpose(np.min(pos_pos_ratio_matrix, axis=0))
        pos_pos_ratio_max = np.transpose(np.max(pos_pos_ratio_matrix, axis=0))
        pos_neg_ratio_min = np.transpose(np.min(pos_neg_ratio_matrix, axis=0))
        pos_neg_ratio_max = np.transpose(np.max(pos_neg_ratio_matrix, axis=0))
        neg_neg_ratio_min = np.transpose(np.min(neg_neg_ratio_matrix, axis=0))
        neg_neg_ratio_max = np.transpose(np.max(neg_neg_ratio_matrix, axis=0))
        neg_pos_ratio_min = np.transpose(np.min(neg_pos_ratio_matrix, axis=0))
        neg_pos_ratio_max = np.transpose(np.max(neg_pos_ratio_matrix, axis=0))

        self.features = np.c_[self.features, pos_pos_ratio_min, pos_pos_ratio_max, pos_neg_ratio_min, pos_neg_ratio_max,
            neg_neg_ratio_min, neg_neg_ratio_max, neg_pos_ratio_min, neg_pos_ratio_max]

        # print('1, 2, 5, 6', self.features.shape)
        
        # Part 7: Stats for active constraints
        slacks = np.array(branch_instance.get_linear_slacks())
        active_constraints = slacks == 0

        active_matrix = static_features.matrix[active_constraints, :]
        active_matrix = active_matrix[:, candidates].todense()
        count_active_matrix = active_matrix != 0

        # Unit weighting
        unit_sum = np.transpose(np.sum(active_matrix, axis=0))
        unit_mean = np.transpose(np.mean(active_matrix, axis=0))
        unit_std = np.transpose(np.std(active_matrix, axis=0))
        unit_min = np.transpose(np.min(active_matrix, axis=0))
        unit_max = np.transpose(np.max(active_matrix, axis=0))
        unit_count = np.transpose(np.sum(count_active_matrix, axis=0))

        # Add unit weighting features
        self.features = np.c_[self.features, unit_sum, unit_mean, unit_std, unit_min, unit_max, unit_count]
        # print('1, 2, 5, 6, 7a', self.features.shape)
        

        # Inverse sum all weighting
        inverse_sum_all = 1 / static_features.sum_coeffs[active_constraints]
        inverse_sum_all[np.isnan(inverse_sum_all)] = 0
        inverse_sum_all[np.isinf(inverse_sum_all)] = 0
        inverse_sum_all_matrix = np.multiply(active_matrix, inverse_sum_all)
        count_inverse_sum_all_matrix = np.multiply(count_active_matrix, inverse_sum_all)

        inv_sum_all_sum = np.transpose(np.sum(inverse_sum_all_matrix, axis=0))
        inv_sum_all_mean = np.transpose(np.mean(inverse_sum_all_matrix, axis=0))
        inv_sum_all_std = np.transpose(np.std(inverse_sum_all_matrix, axis=0))
        inv_sum_all_min = np.transpose(np.min(inverse_sum_all_matrix, axis=0))
        inv_sum_all_max = np.transpose(np.max(inverse_sum_all_matrix, axis=0))
        inv_sum_all_count = np.transpose(np.sum(count_inverse_sum_all_matrix, axis=0))

        # Add inverse sum all weighting features
        self.features = np.c_[self.features, inv_sum_all_sum, inv_sum_all_mean, inv_sum_all_std, inv_sum_all_min, inv_sum_all_max, inv_sum_all_count]
        # print('1, 2, 5, 6, 7b', self.features.shape)
        
        # Inverse sum candidate weighting
        inverse_sum_candidate = 1 / np.sum(active_matrix, axis=1)
        inverse_sum_candidate[np.isnan(inverse_sum_candidate)] = 0
        inverse_sum_candidate[np.isinf(inverse_sum_candidate)] = 0
        inverse_sum_candidate_matrix = np.multiply(active_matrix, inverse_sum_candidate)
        count_inverse_sum_candidate_matrix = np.multiply(count_active_matrix, inverse_sum_candidate)

        inv_sum_candidate_sum = np.transpose(np.sum(inverse_sum_candidate_matrix, axis=0))
        inv_sum_candidate_mean = np.transpose(np.mean(inverse_sum_candidate_matrix, axis=0))
        inv_sum_candidate_std = np.transpose(np.std(inverse_sum_candidate_matrix, axis=0))
        inv_sum_candidate_min = np.transpose(np.min(inverse_sum_candidate_matrix, axis=0))
        inv_sum_candidate_max = np.transpose(np.max(inverse_sum_candidate_matrix, axis=0))
        inv_sum_candidate_count = np.transpose(np.sum(count_inverse_sum_candidate_matrix, axis=0))

        self.features = np.c_[static_features.features[candidates, :], self.features, inv_sum_candidate_sum, inv_sum_candidate_mean, inv_sum_candidate_std, inv_sum_candidate_min, inv_sum_candidate_max, inv_sum_candidate_count]

In [7]:
class LoggingCallback(CPX_CB.MIPInfoCallback):    
    def __call__(self):
        nc = self.get_num_nodes()
        if nc > self.num_nodes:
            self.num_nodes = nc
            
        self.total_time = self.get_time() - self.get_start_time()

In [8]:
class VariableSelectionCallback(CPX_CB.BranchCallback):     
    def create_default_branches(self):
        for branch in range(self.get_num_branches()):
            self.make_cplex_branch(branch)
            
    def get_data(self):
        node_data = self.get_node_data()
        if node_data is None:
            node_data = {}
            node_data['branch_history'] = []

        return node_data

    def get_clone(self):
        cclone = CPX.Cplex(self.c)

        node_data = self.get_data()
        apply_branch_history(cclone, node_data['branch_history'])

        return cclone
            
    def get_candidates(self):
        ###################################################
        # TODO: Check if PC are weighted by fractionality
        ###################################################
        ps_costs = self.get_pseudo_costs(self.var_idx_lst)
        soln = self.get_values(self.var_idx_lst)    
        
        scores = [(ps[0]*ps[1], s, v) for ps, s, v in zip(ps_costs, soln, self.var_idx_lst)]
        scores = sorted(scores, key=itemgetter(0), reverse=True)
        
        candidates = []
        ps_scores = []
        for pss, s, v in scores:
            # Ignore the candidate variable if its already an integer
            if not (abs(s - round(s)) <= self.EPSILON):
                candidates.append(v)
                ps_scores.append(pss)
                
            if len(candidates) == self.K:
                break
            
        return candidates, ps_scores
    
    
    def get_branch_solution(self, cclone, cand, bound_type):
        cand_val = self.get_values(cand)
        
        get_bounds = None
        set_bounds = None
        new_bound = None
        if bound_type == LOWER_BOUND:
            get_bounds = self.get_lower_bounds
            set_bounds = cclone.variables.set_lower_bounds
            new_bound = floor(cand_val) + 1
        elif bound_type == UPPER_BOUND:
            get_bounds = self.get_upper_bounds
            set_bounds = cclone.variables.set_upper_bounds
            new_bound = floor(cand_val)

        original_bound = get_bounds(cand)

        set_bounds(cand, new_bound)
        status, objective = solve_as_lp(cclone, max_iterations=50)
        set_bounds(cand, original_bound)

        return status, objective

    def get_strong_branching_score(self, candidates):
        cclone = self.get_clone()
        status, parent_objval = solve_as_lp(cclone)
        
        sb_scores = []
        for cand in candidates:
            _, lower_objective = self.get_branch_solution(cclone, cand, self.LOWER_BOUND)
            _, upper_objective = self.get_branch_solution(cclone, cand, self.UPPER_BOUND)

            delta_lower = max(lower_objective-parent_objval, self.EPSILON)
            delta_upper = max(upper_objective-parent_objval, self.EPSILON)

            sb_score = delta_lower*delta_upper
            sb_scores.append(sb_score)

        return np.asarray(sb_scores) 
            
    def __call__(self):
        self.times_called += 1
        
        if self.times_called == 1:
            disable_cuts(self.c)
                            
        # If strategy is SB_DEFAULT or PC_DEFAULT then shortcut
        if self.strategy == BS_DEFAULT:
            self.c.parameters.mip.strategy.variableselect.set(0)
            self.create_default_branches()
            return
        
        elif self.strategy == BS_SB:
            self.c.parameters.mip.strategy.variableselect.set(3)
            self.create_default_branches()
            return
        
        elif self.strategy == BS_PC or (self.strategy == BS_SB_PC and self.times_called > self.THETA):
            self.c.parameters.mip.strategy.variableselect.set(2)
            self.create_default_branches()
            return
        
        # coeff = self.get_objective_coefficients(self.var_lst)
        # print('Objval', objval, sum([cff*sln for cff, sln in zip(coeff, soln)]))
        
        # For all other strategies, collect training data for the first THETA nodes
        if self.times_called <= self.THETA:            
            # Get branching candidates based on pseudo costs
            candidates, ps_scores = self.get_candidates()
            if len(candidates) == 0:
                # Nothing to branch
                return

            # Calculate SB scores for branching candidates
            sb_scores = self.get_strong_branching_score(candidates)        
            branch_var = candidates[np.argmax(sb_scores)]
            
            # Prepare training data            
            dfobj = DynamicFeatures(self, self.stat_feat, candidates)
            print(dfobj.features.shape)
            
            if self.times_called == self.THETA:
                # Train model
                pass
            
        else:
            branch_var = 0
            
            
        branch_val = self.get_values(branch_var)
        objval = self.get_objective_value()
        node_data = self.get_data()
        branches = [(branch_var, LOWER_BOUND, floor(branch_val) + 1), 
                    (branch_var, UPPER_BOUND, floor(branch_val))]

        for branch in branches:
            node_data_clone = node_data.copy()
            node_data_clone['branch_history'] = node_data['branch_history'][:]
            node_data_clone['branch_history'].append(branch)

            self.make_branch(objval, variables=[branch], constraints=[], node_data=node_data_clone)

        if self.times_called == 1:
            self.abort()

In [9]:
def solve_instance(path='set_cover.lp', 
                   branch_strategy=BS_SB_PC,
                   primal_bound=None,
                   theta=THETA,
                   k=K,
                   epsilon=EPSILON,
                   upper_bound=UPPER_BOUND,
                   lower_bound=LOWER_BOUND):
    
    # Read instance and set default parameters
    c = CPX.Cplex(path)
    set_params(c, primal_bound=primal_bound, test=True)

    stat_feat = StaticFeatures(c)
    print('Stat feat shape', stat_feat.features.shape)
    var_lst = c.variables.get_names()
    var_idx_lst = c.variables.get_indices(var_lst)

    log_cb = c.register_callback(LoggingCallback)
    log_cb.num_nodes = 0
    log_cb.total_time = 0
    
    vsel_cb = c.register_callback(VariableSelectionCallback)
    vsel_cb.c = c
    vsel_cb.var_lst = var_lst
    vsel_cb.var_idx_lst = var_idx_lst
    vsel_cb.stat_feat = stat_feat
    vsel_cb.strategy = branch_strategy
    vsel_cb.times_called = 0
    vsel_cb.THETA = theta
    vsel_cb.K = k
    vsel_cb.EPSILON = epsilon
    vsel_cb.UPPER_BOUND = upper_bound
    vsel_cb.LOWER_BOUND = lower_bound
    
    # Solve the instance and save stats
    c.solve()
    status = c.solution.get_status()
    print(status, log_cb.num_nodes, log_cb.total_time)
    
    return c

In [10]:
c = solve_instance()

Stat feat shape (1000, 18)
Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_MIP_Strategy_HeuristicEffort            0


Tried aggregator 1 time.
MIP Presolve eliminated 177 rows and 9 columns.
Reduced MIP has 323 rows, 991 columns, and 16063 nonzeros.
Reduced MIP has 991 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (5.80 ticks)
Probing time = 0.00 sec. (0.21 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 323 rows, 991 columns, and 16063 nonzeros.
Reduced MIP has 991 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.03 sec. (5.52 ticks)
Probing time = 0.00 sec. (0.21 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.03 sec. (12.29 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variable B NodeID Parent  Depth

      0     0      189.6274    76                    189.6274      267         
      0     0      

/home/rahul/miniconda3/envs/pyCPLEX/lib/python3.7/site-packages/ipykernel_launcher.py:82: RuntimeWarning: divide by zero encountered in true_divide
/home/rahul/miniconda3/envs/pyCPLEX/lib/python3.7/site-packages/ipykernel_launcher.py:82: RuntimeWarning: invalid value encountered in true_divide
/home/rahul/miniconda3/envs/pyCPLEX/lib/python3.7/site-packages/ipykernel_launcher.py:85: RuntimeWarning: invalid value encountered in true_divide
/home/rahul/miniconda3/envs/pyCPLEX/lib/python3.7/site-packages/ipykernel_launcher.py:146: RuntimeWarning: divide by zero encountered in true_divide
